<a href="https://colab.research.google.com/github/MamMates/ml-food-price/blob/main/MamMates_Food_Price.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import

In [188]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

## Data Preprocessing

In [189]:
dataset_link = "https://docs.google.com/spreadsheets/d/1e_aUUmqyBmFP15BlJCKY-MHv5YFRTRROBEjk1ArDT84"

df_dataset = pd.read_csv(f'{dataset_link}/export?gid=261629379&format=csv')
df_dataset

,name,province,environment,rating,price
0,roti cokelat,17.0,2.0,3.0,"Rp11,000"
1,roti kukus cokelat,17.0,2.0,3.0,"Rp15,000"
2,roti panggang cokelat,17.0,2.0,3.0,"Rp15,000"
3,roti bakar cokelat,17.0,2.0,3.0,"Rp23,500"
4,roti tawar,17.0,2.0,3.0,"Rp26,000"
...,...,...,...,...,...
348,roti keju,16.0,3.0,3.0,"Rp5,400"
349,roti tawar,16.0,3.0,3.0,"Rp16,000"
350,roti tawar,16.0,3.0,3.0,"Rp25,000"
351,roti kukus srikaya,16.0,3.0,3.0,"Rp7,000"


In [190]:
df_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 353 entries, 0 to 352
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   name         342 non-null    object 
 1   province     342 non-null    float64
 2   environment  342 non-null    float64
 3   rating       339 non-null    float64
 4   price        335 non-null    object 
dtypes: float64(3), object(2)
memory usage: 13.9+ KB


In [191]:
df_dataset.isna().sum()

name           11
province       11
environment    11
rating         14
price          18
dtype: int64

In [192]:
def clean_dataset(df):
  df.dropna(how='all', inplace=True)
  df['rating'].ffill(inplace=True)
  df['rating'] = df['rating'].astype(np.int32)
  df['environment'] = df['environment'].astype(np.int32)

  df['price'] = df['price'].replace('[^\d]', '', regex=True)
  df['price'] = df['price'].replace('', np.nan)
  df['price'] = df['price'].astype(np.float32)
  df['price'].ffill(inplace=True)

  df['province'] = df['province'].astype(np.int32)
  return df


In [193]:
df_dataset = clean_dataset(df_dataset)
df_dataset

,name,province,environment,rating,price
0,roti cokelat,17,2,3,11000.0
1,roti kukus cokelat,17,2,3,15000.0
2,roti panggang cokelat,17,2,3,15000.0
3,roti bakar cokelat,17,2,3,23500.0
4,roti tawar,17,2,3,26000.0
...,...,...,...,...,...
348,roti keju,16,3,3,5400.0
349,roti tawar,16,3,3,16000.0
350,roti tawar,16,3,3,25000.0
351,roti kukus srikaya,16,3,3,7000.0


In [194]:
df_salary = pd.read_csv(f'{dataset_link}/export?gid=533543368&format=csv', index_col='id')
df_salary.head()

,province,salary
id,,
1,Nanggroe Aceh Darussalam (Ibu Kota Banda Aceh),3.413.666
2,Sumatera Utara (Ibu Kota Medan),2.710.493
3,Sumatera Selatan (Ibu Kota Palembang),3.404.177
4,Sumatera Barat (Ibu Kota Padang),2.742.476
5,Bengkulu (Ibu Kota Bengkulu),2.418.280


In [195]:
df_salary['salary'] = df_salary['salary'].replace('[^\d]', '', regex=True)
df_salary['salary'] = df_salary['salary'].astype(np.float32)

In [196]:
df_salary.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 38 entries, 1 to 38
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   province  38 non-null     object 
 1   salary    38 non-null     float32
dtypes: float32(1), object(1)
memory usage: 760.0+ bytes


In [197]:
df_merged = pd.merge(df_dataset, df_salary, left_on='province', right_on='id', how='inner')
df_merged

,name,province_x,environment,rating,price,province_y,salary
0,roti cokelat,17,2,3,11000.0,DKI Jakarta (Ibu Kota Jakarta),4900798.0
1,roti kukus cokelat,17,2,3,15000.0,DKI Jakarta (Ibu Kota Jakarta),4900798.0
2,roti panggang cokelat,17,2,3,15000.0,DKI Jakarta (Ibu Kota Jakarta),4900798.0
3,roti bakar cokelat,17,2,3,23500.0,DKI Jakarta (Ibu Kota Jakarta),4900798.0
4,roti tawar,17,2,3,26000.0,DKI Jakarta (Ibu Kota Jakarta),4900798.0
...,...,...,...,...,...,...,...
337,roti keju,16,3,3,5400.0,Banten (Ibu Kota Serang),2661280.0
338,roti tawar,16,3,3,16000.0,Banten (Ibu Kota Serang),2661280.0
339,roti tawar,16,3,3,25000.0,Banten (Ibu Kota Serang),2661280.0
340,roti kukus srikaya,16,3,3,7000.0,Banten (Ibu Kota Serang),2661280.0


In [198]:
df_merged.drop(columns=['province_x', 'province_y'], inplace=True)

In [199]:
df_merged.head()

,name,environment,rating,price,salary
0,roti cokelat,2,3,11000.0,4900798.0
1,roti kukus cokelat,2,3,15000.0,4900798.0
2,roti panggang cokelat,2,3,15000.0,4900798.0
3,roti bakar cokelat,2,3,23500.0,4900798.0
4,roti tawar,2,3,26000.0,4900798.0


In [200]:
unique_words = set(' '.join(df_merged['name']).split())

for word in unique_words:
    df_merged[word] = df_merged['name'].apply(lambda x: 1 if word in x else 0)

df_merged.drop('name', axis=1, inplace=True)

In [201]:
df_merged

,environment,rating,price,salary,panggang,bakar,tawar,cokelat,srikaya,kukus,keju,roti
0,2,3,11000.0,4900798.0,0,0,0,1,0,0,0,1
1,2,3,15000.0,4900798.0,0,0,0,1,0,1,0,1
2,2,3,15000.0,4900798.0,1,0,0,1,0,0,0,1
3,2,3,23500.0,4900798.0,0,1,0,1,0,0,0,1
4,2,3,26000.0,4900798.0,0,0,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
337,3,3,5400.0,2661280.0,0,0,0,0,0,0,1,1
338,3,3,16000.0,2661280.0,0,0,1,0,0,0,0,1
339,3,3,25000.0,2661280.0,0,0,1,0,0,0,0,1
340,3,3,7000.0,2661280.0,0,0,0,0,1,1,0,1


## Data Analysist

In [202]:
df_merged.head()

,environment,rating,price,salary,panggang,bakar,tawar,cokelat,srikaya,kukus,keju,roti
0,2,3,11000.0,4900798.0,0,0,0,1,0,0,0,1
1,2,3,15000.0,4900798.0,0,0,0,1,0,1,0,1
2,2,3,15000.0,4900798.0,1,0,0,1,0,0,0,1
3,2,3,23500.0,4900798.0,0,1,0,1,0,0,0,1
4,2,3,26000.0,4900798.0,0,0,1,0,0,0,0,1


In [203]:
df_merged['environment'].value_counts()

2    88
3    86
0    84
1    84
Name: environment, dtype: int64

## Modeling

In [204]:
from sklearn.model_selection import train_test_split
X = df_merged.drop(columns='price')
y = df_merged['price']

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [205]:
X_train.shape, X_test.shape

((273, 11), (69, 11))

In [206]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [207]:
from keras.layers import Dense, Dropout
from keras import Sequential, Input

model = Sequential([
  Dense(512, activation='relu', input_shape=(X_train.shape[1],)),
  Dropout(0.5),
  Dense(256,activation='relu'),
  Dropout(0.5),
  Dense(126,activation='relu'),
  Dropout(0.5),
  Dense(64,activation='relu'),
  Dropout(0.5),
  Dense(32,activation='relu'),
  Dropout(0.5),
  Dense(1, activation='linear')
])

model.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_48 (Dense)            (None, 512)               6144      
                                                                 
 dropout_40 (Dropout)        (None, 512)               0         
                                                                 
 dense_49 (Dense)            (None, 256)               131328    
                                                                 
 dropout_41 (Dropout)        (None, 256)               0         
                                                                 
 dense_50 (Dense)            (None, 126)               32382     
                                                                 
 dropout_42 (Dropout)        (None, 126)               0         
                                                                 
 dense_51 (Dense)            (None, 64)               

In [208]:
model.compile(optimizer='adam',loss='mse', metrics=['mae'])

## Train

In [209]:
from keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_mae',
                               mode='min',
                               patience=100,
                               restore_best_weights=True,
                               verbose=1
)

In [210]:
history = model.fit(
    X_train, y_train.values,
    validation_data=(X_test,y_test.values),
    epochs=1000,
    callbacks=[early_stopping]
)

Epoch 1/1000
9/9 [==============================] - 11s 34ms/step - loss: 329305184.0000 - mae: 16568.0234 - val_loss: 326839296.0000 - val_mae: 16562.5898
Epoch 2/1000
9/9 [==============================] - 0s 15ms/step - loss: 329229408.0000 - mae: 16565.8125 - val_loss: 326679328.0000 - val_mae: 16557.8379
Epoch 3/1000
9/9 [==============================] - 0s 12ms/step - loss: 328868064.0000 - mae: 16554.8418 - val_loss: 325806272.0000 - val_mae: 16531.8672
Epoch 4/1000
9/9 [==============================] - 0s 12ms/step - loss: 326845888.0000 - mae: 16494.9062 - val_loss: 321412288.0000 - val_mae: 16400.6621
Epoch 5/1000
9/9 [==============================] - 0s 12ms/step - loss: 317706848.0000 - mae: 16224.4561 - val_loss: 302826112.0000 - val_mae: 15833.8691
Epoch 6/1000
9/9 [==============================] - 0s 14ms/step - loss: 280748320.0000 - mae: 15077.2393 - val_loss: 238641632.0000 - val_mae: 13695.2725
Epoch 7/1000
9/9 [==============================] - 0s 13ms/step - lo